<a href="https://colab.research.google.com/github/HernanJMaurice/androidSensors/blob/main/HPC/Maurice_Hernan_Ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [13]:
pip install requests

In [24]:
import requests
import json

url = "https://fcm.googleapis.com/fcm/send"

data = {
    "to":"/topics/all",
    "notification":{
        "sound": "default",
        "body" : "Este es el cuerpo",
        "title": "Este es el Titulo",
        "content_available" : True,
        "priority" : "high"
    },
    "data":{
        "sound": "default",
        "body" :"Este es el cuerpo",
        "title" :"Este es el Titulo",
        "content_available": True,
        "priority" :"high"
    }
}

headers = {'Authorization': 'key=AAAACPWgRFU:APA91bEK8Eo-3Mdwr_hzjj-rVv4D4KAUPBzCA0Uot_UdkOqL_zR2Op8M5gYNX3txi7tWotiudXuBtZtULhgai0rZmdSHEiAGw4ZorFSq37nPupC1wS8ckti3UYrN3wpvby9ljSPCq-Rq', 'Content-Type': 'application/json'}
data=json.dumps(data)
r = requests.post(url, data, headers=headers)
print(r.content)

b'{"message_id":7885713075462401223}'


In [12]:
!mpicc -o hello hello.c

hello.c: In function ‘main’:
hello.c:18:25: warning: missing terminating " character
     char *message_fmt = "POST /apikey=%s&command=%s HTTP/1.0
                         ^
hello.c:18:25: error: missing terminating " character
     char *message_fmt = "POST /apikey=%s&command=%s HTTP/1.0
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
hello.c:20:1: warning: missing terminating " character
 ";
 ^
hello.c:20:1: error: missing terminating " character
 ";
 ^~
hello.c:22:5: error: expected expression before ‘struct’
     struct hostent *server;
     ^~~~~~
hello.c:31:12: warning: missing terminating " character
     printf("Request:
            ^
hello.c:31:12: error: missing terminating " character
     printf("Request:
            ^~~~~~~~~
hello.c:32:1: error: expected expression before ‘%’ token
 %s
 ^
hello.c:33:1: warning: missing terminating " character
 ",message);
 ^
hello.c:33:1: error: missing terminating " character
 ",message);
 ^~~~~~~~~~~
hello.c:85:12: warning

In [ ]:
!mpirun --allow-run-as-root -n 1 hello